<a href="https://colab.research.google.com/github/arinajman/personal/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtener datos del US Treasury Bill a 1 año

In [1]:
!pip install fredapi

In [2]:
import pandas as pd
from pandas_datareader import data as pdr
import datetime
from fredapi import Fred
import os

In [3]:
#help(os)
#dir(os)
#help(os.environ)

In [4]:
# Establecer la variable de entorno (solo para la sesión actual)
os.environ['API_KEY'] = '' #'TU_CLAVE_SECRETA'
# Acceder a la variable de entorno en tu código
api_key = os.environ.get('API_KEY')

if api_key:
    print(f"La clave API es: {api_key[:5]}...") # Muestra solo los primeros caracteres por seguridad
    # Utiliza api_key en tus llamadas a la API
else:
    print("La clave API no está configurada como variable de entorno.")

La clave API es: cd3a4...


In [5]:
#help(pdr)
#dir(pdr)
#help(pdr.DataReader)
#dir(pdr.DataReader)

In [6]:
#help(Fred)
#dir(Fred)

In [7]:
fred=Fred(api_key)
fred

In [8]:
ticker = 'DGS1' #DGS1 is the 1-year US Treasury Bill

In [9]:
fred.get_series_info(ticker)

,0
id,DGS1
realtime_start,2025-05-15
realtime_end,2025-05-15
title,Market Yield on U.S. Treasury Securities at 1-...
observation_start,1962-01-02
observation_end,2025-05-14
frequency,Daily
frequency_short,D
units,Percent
units_short,%


In [10]:
print('observation_start:',fred.get_series_info(ticker)['observation_start'],type(fred.get_series_info(ticker)['observation_start']))
year_start = int(fred.get_series_info(ticker)['observation_start'][:4])
print('year_start:',year_start,type(year_start))
month_start = int(fred.get_series_info(ticker)['observation_start'][5:7])
print('month_start:',month_start,type(month_start))
day_start = int(fred.get_series_info(ticker)['observation_start'][8:10])
print('day_start:',day_start,type(day_start))
start=datetime.datetime(year_start,month_start,day_start)
print('start:',start,type(start))

observation_start: 1962-01-02 <class 'str'>
year_start: 1962 <class 'int'>
month_start: 1 <class 'int'>
day_start: 2 <class 'int'>
start: 1962-01-02 00:00:00 <class 'datetime.datetime'>


In [11]:
df = pdr.DataReader(ticker, 'fred',start)
df

,DGS1
DATE,
1962-01-02,3.22
1962-01-03,3.24
1962-01-04,3.24
1962-01-05,3.26
1962-01-08,3.31
...,...
2025-05-08,4.05
2025-05-09,4.05
2025-05-12,4.11


In [12]:
df_final = df.groupby(df.index.year).last()
df_final = df_final.iloc[:-1]
df_final

,DGS1
DATE,
1962,3.05
1963,3.83
1964,3.99
1965,4.96
1966,5.00
...,...
2020,0.10
2021,0.39
2022,4.73


In [13]:
negative_count = len(df_final[df_final[ticker] < 0])
between_0_1_count = len(df_final[(df_final[ticker] >= 0) & (df_final[ticker] <= 1)])
between_1_3_count = len(df_final[(df_final[ticker] > 1) & (df_final[ticker] <= 3)])
between_3_5_count = len(df_final[(df_final[ticker] > 3) & (df_final[ticker] <= 5)])
between_5_7_count = len(df_final[(df_final[ticker] > 5) & (df_final[ticker] <= 7)])
greater_than_7_count = len(df_final[df_final[ticker] > 7])

print(f"Cantidad de años negativos: {negative_count}")
print(f"Cantidad entre 0 y 1: {between_0_1_count}")
print(f"Cantidad entre 1 y 3: {between_1_3_count}")
print(f"Cantidad entre 3 y 5: {between_3_5_count}")
print(f"Cantidad entre 5 y 7: {between_5_7_count}")
print(f"Cantidad mayor a 7: {greater_than_7_count}")

Cantidad de años negativos: 0
Cantidad entre 0 y 1: 11
Cantidad entre 1 y 3: 7
Cantidad entre 3 y 5: 18
Cantidad entre 5 y 7: 12
Cantidad mayor a 7: 15


In [14]:
df_final.index=df_final.index+1 #porque los intereses aplican al año siguiente
df_final

,DGS1
DATE,
1963,3.05
1964,3.83
1965,3.99
1966,4.96
1967,5.00
...,...
2021,0.10
2022,0.39
2023,4.73


In [15]:
df_final=df_final[:-1] #para solo considerar los años completos
df_final

,DGS1
DATE,
1963,3.05
1964,3.83
1965,3.99
1966,4.96
1967,5.00
...,...
2020,1.59
2021,0.10
2022,0.39


In [16]:
from google.colab import drive
import pickle

drive.mount('/content/drive')

# Asumo que ya tienes tu DataFrame 'df2' como resultado de tus operaciones.
# Si no es así, asegúrate de que la variable 'df2' contenga tu DataFrame.

# Define la ruta donde quieres guardar el DataFrame serializado
ruta_guardado2_pkl = '/content/drive/MyDrive/df2.pkl'

# Serializa y guarda el DataFrame
with open(ruta_guardado2_pkl, 'wb') as archivo:
    pickle.dump(df_final, archivo)

print(f"DataFrame 2 serializado y guardado en: {ruta_guardado2_pkl}")

Mounted at /content/drive
DataFrame 2 serializado y guardado en: /content/drive/MyDrive/df2.pkl
